In [ ]:
%matplotlib inline
import numpy as np
import time
import scipy
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.colors as mpcol
import src.dab_util as du

In [ ]:
import src.signal_gen as sg
reload(sg)
reload(du)

In [ ]:
path_in  = "./input.dat"
path_out = "./output.dat"
a_max = 0.95
n_steps = 64
amps = np.linspace(0.001, a_max, num = n_steps)
txgains = (50, 55, 60, 65, 70, 75, 81, 82, 83, 84, 85, 86, 87, 88, 89)
rxgains = (50, 40, 40, 25, 25, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20)
txgains = (75, 81, 84, 85, 88, 89)
rxgains = (20, 20, 20, 20, 20, 20)
txgains = (83,)
rxgains = (20,)

In [ ]:
from grc.amam_ampm import amam_ampm

In [ ]:
sg.gen_ramps(path="./input.dat", n_periods=64, pause=8, amplitudes=amps)

In [ ]:
top = amam_ampm()

In [ ]:
def extract_measurement(a_in, a_out, db, a_max, n_steps, debug = False):
    a_in  = du.crop_signal(a_in)
    a_out = du.crop_signal(a_out)
    
    if debug:
        plt.plot(np.abs(a_in.real) + 1, color='b');
        plt.plot(np.abs(a_out.real), color='g');
        plt.show()
    
    #l = min(a_out.shape[0], a_in.shape[0])
    #a_out = a_out[0:l]
    #a_in  = a_in[0:l]
    
    #c = du.lagcorr(np.abs(a_out), np.abs(a_in), 120000)[:,0]
    #c = signal.fftconvolve(a_in, a_out) - a_out.shape[0]
    #delay = du.fftlag(np.abs(a_in), np.abs(a_out))
    delay = du.fftlag(a_in, a_out) #TODO
    
    #delay = np.argmax(c)
    a_out = a_out[delay - 1:]
    
    l = min(a_out.shape[0], a_in.shape[0])
    a_out = a_out[0:l]
    a_in  = a_in[0:l]
    
    if debug:
        print ("delay = " + str(delay))
        plt.plot(np.abs(a_in), color='g');
        plt.plot(np.abs(a_out) - 0.5, color='y');
        plt.show()
    
    bins = np.linspace(+0.5/n_steps,a_max + 0.5/n_steps,num=n_steps)
    res = []
    
    a_out_up = scipy.signal.resample(a_out, a_out.shape[0] * 8)
    a_in_up = scipy.signal.resample(a_in, a_in.shape[0] * 8)
    
    a_out_abs = np.abs(a_out_up)
    a_in_abs = np.abs(a_in_up)
    for ampl_1, ampl_2 in zip(bins, bins[1:]):
        res.append(du.get_amp_ratio(ampl_1, ampl_2, a_out_abs, a_in_abs))
    del a_out_abs
    del a_in_abs
    mean_amp, var_amp = zip(*res)
    
    res = []
    for ampl_1, ampl_2 in zip(bins, bins[1:]):
        res.append(du.get_phase(ampl_1, ampl_2, a_out_up, a_in_up))
    mean_phase, var_phase = zip(*res)
    return mean_amp, var_amp, mean_phase, var_phase, db

In [ ]:
reload(du)
res = []

for txgain, rxgain in zip(txgains, rxgains):
    print (txgain, rxgain)
    res_tmp = None
    for i in range(10):
        top.uhd_usrp_sink_0_0.set_gain(txgain)
        top.uhd_usrp_source_0.set_gain(rxgain)
        
        top.file_sink_out.close()
        top.blocks_file_source_0.close()
        
        top.file_sink_out.open(path_out)
        top.blocks_file_source_0.open(path_in, False)
        top.start()
        
        time.sleep(1)
        
        top.stop()
        top.wait()
        
        a_in  = np.fromfile(path_in, dtype=np.complex64)
        a_out = np.fromfile(path_out, dtype=np.complex64)
        res_tmp = extract_measurement(a_in, a_out, txgain, a_max, n_steps, debug=True)
        
        def is_finite(r): return np.all([np.all(np.isfinite(c)) for c in r])
        #def has_small_jumps(mean_amp): return np.max(np.abs(np.diff(mean_amp))) / np.median(np.abs(np.diff(mean_amp))) < 100
        
        #TODO
        if is_finite(res_tmp): # and 1 + has_small_jumps(res_tmp[0]):
            break
        else:
            print (is_finite(res_tmp))#, has_small_jumps(res_tmp[0]))
        
    res.append(res_tmp)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax1 = plt.subplot(211)

def plot_with_label(x, y, color, label):
    ax1.plot(x, y, color=color, label=txgain)
    
for idx, (txgain, rxgain) in enumerate(zip(*(txgains, rxgains))):
    plot_with_label(
        x = amps[1:], 
        y = 10*np.log(res[idx][0])/np.log(10) - rxgain + 102,
        color = mpcol.hsv_to_rgb((idx * 0.75 / len(txgains), 0.6, 1)),
        label = txgain
    )
ax1.set_ylabel("Gain [dB]")

ax2 = plt.subplot(212)

def plot_with_label(x, y, color, label):
    ax2.plot(x, y, color=color, label=txgain)
    
for idx, (txgain, rxgain) in enumerate(zip(*(txgains, rxgains))):
    plot_with_label(
        x = amps[1:],
        y = res[idx][2],
        color = mpcol.hsv_to_rgb((idx * 0.75 / len(txgains), 0.6, 1)),
        label = txgain
    )

ax2.set_ylabel("Pase [degree]")
ax2.set_xlabel("Amplitude")

#legend
# Shrink current axis by 20%
box = ax1.get_position()
ax1.set_position([box.x0, box.y0, box.width * 0.8, box.height])
box = ax2.get_position()
ax2.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax1.legend(loc='center left', bbox_to_anchor=(1.05, -0.3))


plt.show()